### Imports

In [27]:
import pandas as pd
import re
import os

# Reading and wrangling the data

In [2]:
## fetching the data
df_citrusdal = pd.read_csv('https://raw.githubusercontent.com/juelha/IANNWTF_FINAL/main/data/citrus%20data/Citrusdal/Citrusdal_cultivars.csv?token=GHSAT0AAAAAABSUU62YXD3HSSJF6KMXH5VAYSNRVSQ')
df_east_cape = pd.read_csv('https://raw.githubusercontent.com/juelha/IANNWTF_FINAL/main/data/citrus%20data/Eastern%20-%20Cape/Eastern_Cape_cultivars.csv?token=GHSAT0AAAAAABSUU62YUQ3SZ3BY3ZD54JLSYSNRWDA')
df_limpopo = pd.read_csv('https://raw.githubusercontent.com/juelha/IANNWTF_FINAL/main/data/citrus%20data/Letsitele/Letsitele_Valnecia_cultivars.csv?token=GHSAT0AAAAAABSUU62ZIIQARLYH4TUYL5KSYSNRX7A')

# add the production region to the letsitele data set
df_limpopo['Production region'] = 'Limpopo'

# uniform naming for the seasonal data
df_citrusdal.rename(columns = {'Year':'season'}, inplace = True)
df_east_cape.rename(columns = {'Year':'season'}, inplace = True)

# merge the relevant data sets for easier wrangling
df_merged = pd.concat([df_citrusdal, df_east_cape, df_limpopo])

# columns are renamed for easier handling and to remove the new lines
df_merged.rename(columns = {'Production region':'region'}, inplace = True)

# get columns and remove new lines from it, then set as new columns
old_columns = df_merged.columns.values.tolist()
df_merged.columns = [re.sub('\n', ' ', column) for column in old_columns]

In [3]:
# dropping unneccesary columns
df_merged = df_merged.drop(columns = ['Farm', 'Block', 'Rootstock', 'plant year',
                                      'Age', 'Na (mg/kg)', 'S (%)', 'Cl (%)',
                                      'Cu (mg/kg)', 'Fe (mg/kg)', 'Mn (mg/kg)',
                                      'Zn (mg/kg)', 'B (mg/kg)', 'Mo (mg/kg)'])
## missing values
# drop columns that have na values for production or cultivar or season
df_merged.dropna(subset=['production (ton/ha)', 'Cultivar', 'season'], inplace = True)

# drop columns that have missing values for ALL macro nutrients
df_merged = df_merged.dropna(subset=['N (%)', 'P (%)', 'K (%)', 'Ca (%)', 'Mg (%)'], how='all')

# merge all Navel, Swartvlei and Midknight subclasses of cultivars
df_merged['Cultivar'] = df_merged['Cultivar'].str.replace('^Navel.*', 'Navel (all)', regex=True)
df_merged['Cultivar'] = df_merged['Cultivar'].str.replace('^Swartvlei.*', 'Swartvlei (all)', regex=True)
df_merged['Cultivar'] = df_merged['Cultivar'].str.replace('^Midknight.*', 'Midknight', regex=True)

df_merged = df_merged.replace(['Addo (SRV)', 'CT Miller', 'Kirkwood (SRV)', 'Sondagsriver'],
                              ['Eastern Cape', 'Eastern Cape', 'Eastern Cape', 'Eastern Cape'])

# create Species column based on cultivars then drop cultivars
# list of cultivars and their species
val_oranges = ['Midknight', 'Val Midnight', 'Val Delta', 'Delta', 'Valencia', 'Bennie', 'Lavalle', 'Swartvlei (all)']
navel_oranges = ['Palmer', 'Navel (all)', 'Autumn Gold', 'Washington ']
mandarins = ['Nova', 'Nadorcott', 'ORRI']

# creating the new column 'Species' based on the lists
df_merged = df_merged[df_merged['Cultivar'].isin(navel_oranges) == False]
df_merged = df_merged[df_merged['Cultivar'].isin(mandarins) == False]
df_merged.loc[df_merged['Cultivar'].isin(val_oranges) == True, 'species'] = 'Valencia Orange'
df_merged.drop('Cultivar', axis=1, inplace=True)
df_merged.drop('species', axis=1, inplace=True)

# remove numerical 'production (ton/ha)' column and add 'yield' that categorizes the yield in high or low (current boundary = 50)
df_merged.loc[df_merged['production (ton/ha)'] > 50, 'yield'] = 1
df_merged.loc[df_merged['production (ton/ha)'] <= 50, 'yield'] = 0
df_merged.drop('production (ton/ha)', axis=1, inplace=True)

In [29]:
# split the wrangled data frame by their region and drop the column afterwards
df_citrusdal = df_merged[df_merged['region'] == 'Citrusdal']
df_citrusdal.drop('region', axis=1, inplace=True)

df_limpopo = df_merged[df_merged['region'] == 'Limpopo']
df_limpopo.drop('region', axis=1, inplace=True)

df_east_cape = df_merged[df_merged.iloc[:,0].isin(['Citrusdal', 'Limpopo']) == False]
df_east_cape.drop('region', axis=1, inplace=True)

# reset and drop index
df_merged.reset_index(drop=True, inplace=True)
df_citrusdal.reset_index(drop=True, inplace=True)
df_limpopo.reset_index(drop=True, inplace=True)
df_east_cape.reset_index(drop=True, inplace=True)

## exporting dataframes
# getting current folder 
current_folder = str(globals()['_dh'][0])

# exporting to csv
df_merged.to_csv(str(current_folder + '/../data/' + 'df_merged.csv'), index=False)  
df_citrusdal.to_csv(str(current_folder + '/../data/citrus_data/Citrusdal/' + 'df_citrusdal.csv'), index=False) 
df_east_cape.to_csv(str(current_folder + '/../data/citrus_data/Eastern_Cape/' + 'df_east_cape.csv'), index=False) 
df_limpopo.to_csv(str(current_folder + '/../data/citrus_data/Letsitele/' + 'df_limpopo.csv'), index=False) 
